In [182]:
# !pip install --upgrade pip
# !pip install bs4 selenium black pandas --quiet

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

In [6]:
sites = {
    "yahoo": {
        "base_url": "https://finance.yahoo.com/topic/",
        "category": {
            "economics": "economic-news",
            "tech": "tech",
            "housing": "housing-market",
        },
    }
}

In [43]:
req = requests.get(sites["yahoo"]["base_url"] + sites["yahoo"]["category"]["economics"])
soup = BeautifulSoup(req.content, 'html.parser')
# article_titles = [t.text for t in soup.find_all(class_="Mb(5px)")]
# links = [t.find('a')["href"] for t in soup.find_all(class_="Mb(5px)")]
titles = soup.find_all(class_="Mb(5px)")
title_text = [t.text for t in titles]
links = [t.find("a")["href"] for t in titles]

In [44]:
title_text

['Oil Set to Eke Out Weekly Gain as Supply and Inflation Set Tone',
 'Europe’s Economy Needs to Grow More Quickly, Donohoe Says',
 'Stocks Churn as Traders Pare Back Fed Rate Bets: Markets Wrap',
 'Oil set for weekly gain on hopes of firmer demand',
 'Suriname’s Oil Resources Seen Totaling 2.4 Billion Barrels',
 'Fed Officials Suggest Interest Rates Should Stay High for Longer',
 'Petro Meets Colombian Central Bankers He Blamed for Weak Economy',
 'Firefighters Gain Control of One Blaze Threatening Oil Sands in Alberta',
 'Oil Rises as Shrinking US Stockpiles Add to Risk-On Sentiment',
 "US proposes ending new federal leases in nation's biggest coal region"]

In [13]:
links

['https://finance.yahoo.com/news/asian-stocks-drop-traders-rethink-224525599.html',
 'https://finance.yahoo.com/news/oil-set-eke-weekly-gain-232819911.html',
 'https://finance.yahoo.com/news/oil-set-weekly-gain-signs-010613579.html',
 'https://finance.yahoo.com/news/suriname-oil-resources-seen-totaling-223930531.html',
 'https://finance.yahoo.com/news/fed-mester-suggests-interest-rates-160000693.html',
 'https://finance.yahoo.com/news/petro-meets-colombian-central-bankers-220147115.html',
 'https://finance.yahoo.com/news/firefighters-gain-control-one-blaze-195526122.html',
 'https://finance.yahoo.com/news/oil-ticks-higher-us-stockpile-232703377.html',
 'https://finance.yahoo.com/news/us-proposes-ending-federal-leases-174509228.html',
 'https://finance.yahoo.com/news/banks-continue-become-increasingly-less-173404596.html']

In [ ]:
def parse_articles_bs(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')    
    title_text = [t.text for t in soup.find_all(class_="Mb(5px)")]
    links = [t.find('a')["href"] for t in soup.find_all(class_="Mb(5px)")]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [178]:
def parse_articles_selenium(article_list):
    titles = article_list.find_elements(by=By.CLASS_NAME, value="sa_text")
    title_text = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").text for t in titles]
    links = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").get_attribute("href") for t in titles]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [15]:
def get_article(link: str):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')
    date = soup.select_one("#caas-art-9dffa534-bcd1-3e24-8377-ba1fb9d5347e > article > div > div > div > div > div > div > div:nth-child(1) > div.caas-content-byline-wrapper > div.caas-attr > div > div.caas-attr-time-style > time")["datetime"]
    content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])
    return {"date": date, "content": content}

In [201]:
req = requests.get("https://news.naver.com/breakingnews/section/101/259")
soup = BeautifulSoup(req.content, 'html.parser')
titles = [t.text.strip('\n') for t in soup.find_all(class_="sa_text_title")]
links = [t['href'] for t in soup.find_all(class_="sa_text_title")]

In [210]:
datetime.date()

datetime.datetime(2024, 5, 17, 15, 59, 24, 340080)

In [47]:
link = 'https://finance.yahoo.com/news/asian-stocks-drop-traders-rethink-224525599.html'
req = requests.get(link)
soup = BeautifulSoup(req.content, 'html.parser')
date = soup.find("time")["datetime"]
content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])

In [48]:
date

'2024-05-17T05:56:36.000Z'

In [42]:
print(content)

(Bloomberg) -- European stocks headed for an early decline alongside Asian equities as signs of weakness in China and lower US rate cut forcasts weighed on risk sentiment.
Most Read from Bloomberg
US Inflation Data Was Accidentally Released 30 Minutes Early
With a BlackRock CEO, $9 Trillion Vanguard Braces for Turbulence
Putin and Xi Vow to Step Up Fight to Counter US ‘Containment’
Jamie Dimon Sees ‘Lot of Inflationary Forces in Front of Us’
Dow Average Touches 40,000 Before Pulling Back: Markets Wrap
Contracts for the Euro Stoxx 50 fell, following Thursday declines for US stocks and a downbeat day in Asian equity markets that dragged a gauge of the region’s shares lower for the first time in six sessions.
The moves followed a lackluster day on Wall Street and further signs of stress in China, where home sales fell in April at a faster pace than the prior month and consumption unexpectedly slowed, providing a new warning sign for the economy.
“A-share investor sentiment dropped notably